In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct

In [2]:
# If using local Docker or remote Qdrant Cloud:
client = QdrantClient(url="http://localhost:6333")  

# Create a collection named "my_collection" with 1536-dim vectors using cosine similarity
client.recreate_collection(
    collection_name="my_collection",
    vectors_config=VectorParams(size=4, distance=Distance.COSINE)
)

/tmp/ipykernel_23634/1790002613.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [3]:
# Example vectors (e.g., embeddings of some texts) and payloads
vectors = [
    [0.05, 0.61, 0.76, 0.74],
    [0.19, 0.81, 0.75, 0.11],
    [0.36, 0.55, 0.47, 0.94],
]
payloads = [
    {"city": "Berlin", "country": "Germany"},
    {"city": "London", "country": "UK"},
    {"city": "Moscow", "country": "Russia"},
]

points = []
for i, (vec, meta) in enumerate(zip(vectors, payloads), start=1):
    points.append(PointStruct(id=i, vector=vec, payload=meta))

# Upsert points into the collection
result = client.upsert(
    collection_name="my_collection",
    points=points,
    wait=True  # wait for operation to complete
)
print(f"Inserted {len(points)} points. Status: {result.status}")

Inserted 3 points. Status: completed


In [4]:
# Query the collection for nearest neighbors of a given vector
query_vector = [0.2, 0.1, 0.9, 0.7]
results = client.search(
    collection_name="my_collection",
    query_vector=query_vector,
    limit=3,            # return 3 nearest points
    with_payload=True   # include payload data in results
)
for point in results:
    print(f"Found ID {point.id} with score {point.score}, payload={point.payload}")

Found ID 1 with score 0.89463294, payload={'city': 'Berlin', 'country': 'Germany'}
Found ID 3 with score 0.83872515, payload={'city': 'Moscow', 'country': 'Russia'}
Found ID 2 with score 0.66603535, payload={'city': 'London', 'country': 'UK'}


/tmp/ipykernel_23634/3024841082.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [5]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

# Find nearest neighbors to the query_vector, but only where city == "London"
results = client.search(
    collection_name="my_collection",
    query_vector=query_vector,
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    limit=3,
    with_payload=True
)
for point in results:
    print(f"Found ID {point.id}, city={point.payload.get('city')}, score={point.score}")

Found ID 2, city=London, score=0.66603535


/tmp/ipykernel_23634/590064084.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [7]:
point.model_dump_json()

'{"id":2,"version":0,"score":0.66603535,"payload":{"city":"London","country":"UK"},"vector":null,"shard_key":null,"order_value":null}'